# Request and process USDA county-level corn data from the API server.

In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
state_alphas = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"]

In [3]:
def tidy(df):
    del df["CV (%)"]
    del df["agg_level_desc"]
    del df["asd_desc"]
    del df["begin_code"]
    del df["class_desc"]
    del df["commodity_desc"]
    del df["congr_district_code"]
    del df["country_code"]
    del df["country_name"]
    del df["domain_desc"]
    del df["domaincat_desc"]
    del df["end_code"]
    del df["freq_desc"]
    del df["group_desc"]
    del df["load_time"]
    del df["location_desc"]
    del df["prodn_practice_desc"]
    del df["reference_period_desc"]
    del df["region_desc"]
    del df["sector_desc"]
    del df["short_desc"]
    del df["source_desc"]
    del df["statisticcat_desc"]
    del df["unit_desc"]
    del df["util_practice_desc"]
    del df["watershed_code"]
    del df["watershed_desc"]
    del df["week_ending"]
    del df["zip_5"]
    del df["county_ansi"]
    del df["state_ansi"]
    
    return df

In [4]:
def get_corn(states, level):
    data = pd.DataFrame()
    for state in states:
        print("Now grabbing... " + state)
        dat = requests.get("http://quickstats.nass.usda.gov/api/api_GET/?key=210BA222-FC6E-3FB2-B4D7-DA2DAA1CC829&source_desc=SURVEY&sector_desc=CROPS&group_desc=FIELD CROPS&commodity_desc=CORN&statisticcat_desc=YIELD&util_practice_desc=GRAIN&unit_desc=BU / ACRE&agg_level_desc=" + level + "&year__GE=1950&state_alpha="+state)
        if dat.status_code == 200:
            print("Data grabbed from USDA successfully...")
            dat = dat.json()
            dat = pd.DataFrame(dat["data"])
            data = pd.concat([data, dat], ignore_index = True)
            print("Filled!")
        else:
            print("Data grabbed from USDA unsuccessfully. Error code " + str(dat.status_code) + ". Skipping.")
    return tidy(data)

In [5]:
def get_corn_area(states, level):
    data = pd.DataFrame()
    for state in states:
        print("Now grabbing... " + state)
        dat = requests.get("http://quickstats.nass.usda.gov/api/api_GET/?key=210BA222-FC6E-3FB2-B4D7-DA2DAA1CC829&source_desc=SURVEY&sector_desc=CROPS&group_desc=FIELD CROPS&commodity_desc=CORN&statisticcat_desc=AREA HARVESTED&util_practice_desc=GRAIN&unit_desc=ACRES&agg_level_desc=" + level + "&year__GE=1950&state_alpha="+state)
        if dat.status_code == 200:
            print("Data grabbed from USDA successfully...")
            dat = dat.json()
            dat = pd.DataFrame(dat["data"])
            data = pd.concat([data, dat], ignore_index = True)
            print("Filled!")
        else:
            print("Data grabbed from USDA unsuccessfully. Error code " + str(dat.status_code) + ". Skipping.")
    return tidy(data)

In [6]:
# Get corn yields
usda_county = get_corn(state_alphas, "COUNTY")

Now grabbing... AL
Data grabbed from USDA successfully...
Filled!
Now grabbing... AK
Data grabbed from USDA unsuccessfully. Error code 400. Skipping.
Now grabbing... AZ
Data grabbed from USDA successfully...
Filled!
Now grabbing... AR
Data grabbed from USDA successfully...
Filled!
Now grabbing... CA
Data grabbed from USDA successfully...
Filled!
Now grabbing... CO
Data grabbed from USDA successfully...
Filled!
Now grabbing... CT
Data grabbed from USDA unsuccessfully. Error code 400. Skipping.
Now grabbing... DE
Data grabbed from USDA successfully...
Filled!
Now grabbing... FL
Data grabbed from USDA successfully...
Filled!
Now grabbing... GA
Data grabbed from USDA successfully...
Filled!
Now grabbing... HI
Data grabbed from USDA unsuccessfully. Error code 400. Skipping.
Now grabbing... ID
Data grabbed from USDA successfully...
Filled!
Now grabbing... IL
Data grabbed from USDA successfully...
Filled!
Now grabbing... IN
Data grabbed from USDA successfully...
Filled!
Now grabbing... IA
Dat

In [7]:
# Get corn harvested areas
usda_county_area = get_corn_area(state_alphas, "COUNTY")

Now grabbing... AL
Data grabbed from USDA successfully...
Filled!
Now grabbing... AK
Data grabbed from USDA unsuccessfully. Error code 400. Skipping.
Now grabbing... AZ
Data grabbed from USDA successfully...
Filled!
Now grabbing... AR
Data grabbed from USDA successfully...
Filled!
Now grabbing... CA
Data grabbed from USDA successfully...
Filled!
Now grabbing... CO
Data grabbed from USDA successfully...
Filled!
Now grabbing... CT
Data grabbed from USDA unsuccessfully. Error code 400. Skipping.
Now grabbing... DE
Data grabbed from USDA successfully...
Filled!
Now grabbing... FL
Data grabbed from USDA successfully...
Filled!
Now grabbing... GA
Data grabbed from USDA successfully...
Filled!
Now grabbing... HI
Data grabbed from USDA unsuccessfully. Error code 400. Skipping.
Now grabbing... ID
Data grabbed from USDA successfully...
Filled!
Now grabbing... IL
Data grabbed from USDA successfully...
Filled!
Now grabbing... IN
Data grabbed from USDA successfully...
Filled!
Now grabbing... IA
Dat

In [8]:
# Get list of counties from USDA file
counties = pd.read_csv("./input/county_district_list.csv")
counties = counties[counties.Use == 1.0]
counties = counties[counties.District != 0]
counties = counties[counties.Name != "State Total"]
counties["State"] = counties["State"].astype(str).str.zfill(2)
counties["County"] = counties["County"].astype(str).str.zfill(3)
counties["District"] = counties["District"].astype(str).str.zfill(2)
counties["GEOID"] = counties["State"] + counties["County"] + counties["District"]

In [9]:
# Get list of county code changes from USDA file
counties_to_change = pd.read_csv("./input/counties_old.csv")

counties_to_change["State"] = counties_to_change["State"].astype(str).str.zfill(2)
counties_to_change["County"] = counties_to_change["County"].astype(str).str.zfill(3)
counties_to_change["District"] = counties_to_change["District"].astype(str).str.zfill(2)
counties_to_change["GEOID"] = counties_to_change["State"] + counties_to_change["County"] + counties_to_change["District"]

counties_to_change["State_new"] = counties_to_change["State_new"].astype(str).str.zfill(2)
counties_to_change["County_new"] = counties_to_change["County_new"].astype(str).str.zfill(3)
counties_to_change["District_new"] = counties_to_change["District_new"].astype(str).str.zfill(2)
counties_to_change["GEOID_new"] = counties_to_change["State_new"] + counties_to_change["County_new"] + counties_to_change["District_new"]

In [10]:
################################################ OTHER (COMBINED) RESULTS ########################################

In [11]:
# Temporarily store "other" results
usda_county["Value"] = usda_county["Value"].astype(float)
usda_other = usda_county.query("county_name == 'OTHER (COMBINED) COUNTIES'").groupby(["state_fips_code", "year", "asd_code"], as_index=True).sum()
usda_other = usda_other.rename(columns = {"Value": "OtherCombined_Value"})

usda_county_area["Value"] = usda_county_area["Value"].str.replace(",", "").astype(float)
usda_other_area = usda_county_area.query("county_name == 'OTHER (COMBINED) COUNTIES'").groupby(["state_fips_code", "year", "asd_code"], as_index = True).sum()
usda_other_area = usda_other_area.rename(columns = {"Value": "OtherSummed_Area"})

In [12]:
# Find how many (current) counties in each agricultural district
county_num =  counties.groupby(["State", "District"], as_index = True).count().drop(columns = ["Name", "Use","GEOID"])
county_num = county_num.rename(columns = {"County": "Count"})

In [13]:
# Find how many (old) counties in each agricultural district
# county_num1 =  counties_to_change.groupby(["State", "District"], as_index = True).count().drop(columns = ["Name","Use","GEOID","Name_new","Use_new","State_new","District_new","County_new"])
# county_num1 = county_num1.rename(columns = {"County": "Count"})

In [14]:
# # Create one dataframe with (old and current) number of counties in each agricultural district
# county_num_all = pd.merge(county_num, county_num1, how = "outer", left_index = True, right_index = True, suffixes = ["_current", "_old"])
# county_num_all = county_num_all.fillna(0)
# county_num_all["count"] = county_num_all.apply(lambda x: x["Count_current"] if x["Count_current"] > 0.0 else x["Count_old"], axis = 1)
# del county_num_all["Count_current"]
# del county_num_all["Count_old"]

In [15]:
# Calculate county areas in each ag district for other (combined) results
usda_other_area = pd.merge(usda_other_area.reset_index(), county_num.reset_index(), left_on = ["state_fips_code", "asd_code"], right_on = ["State","District"], how = "outer")
usda_other_area = usda_other_area.dropna()
usda_other_area["OtherSummed_Area"] = usda_other_area["OtherSummed_Area"] / usda_other_area["Count"]
usda_other_area = usda_other_area.rename(columns = {"OtherSummed_Area": "OtherMean_Area"})
del usda_other_area["State"]
del usda_other_area["District"]
del usda_other_area["Count"]

In [16]:
# Merge into one df
usda_other_all = pd.merge(usda_other_area, usda_other.reset_index(), how = "inner")

In [17]:
################################################ MAIN RESULTS ###################################################

In [18]:
# Process USDA county results
usda_county["GEOID"] = usda_county["state_fips_code"] + usda_county["county_code"] + usda_county["asd_code"]
usda_county.set_index(["state_fips_code","county_code","asd_code","county_name","state_name","state_alpha","GEOID","year"], inplace = True)

usda_county_area["GEOID"] = usda_county_area["state_fips_code"] + usda_county_area["county_code"] + usda_county_area["asd_code"]
usda_county_area = usda_county_area.rename(columns = {"Value": "area"})

In [19]:
# Merge into one df
usda = pd.merge(usda_county.query("county_name != 'OTHER (COMBINED) COUNTIES'"), usda_county_area.query("county_name != 'OTHER (COMBINED) COUNTIES'"), how = "outer", on = ["GEOID","state_fips_code","county_code", "asd_code", "year", "county_name"])
usda = usda.reset_index()
usda["GEOID_new"] = usda["GEOID"]
usda.set_index(["GEOID"], inplace = True)

In [20]:
# Update old GEOIDs to current
for index, row in counties_to_change.iterrows():
    if row["GEOID_new"] != "0000000":
        if row["GEOID"] in usda.index:
            usda.loc[row["GEOID"],"GEOID_new"] = row["GEOID_new"]

In [21]:
# Update indexing
usda.reset_index()
usda["GEOID"] = usda["GEOID_new"]
del usda["GEOID_new"]
del usda["state_alpha"]
del usda["state_name"]
usda["state_fips_code"] = usda["GEOID"].str[:2]
usda["county_code"] = usda["GEOID"].str[2:5]
usda["asd_code"] = usda["GEOID"].str[-2:]
usda.set_index(["state_fips_code","county_code","asd_code","county_name","GEOID","year"], inplace = True)

In [22]:
# Build index structure for final dataframe (including years)
unique_geoids = counties["GEOID"].unique()
years = np.arange(1950, 2019, 1)

unique_geoids = [[unique_geoid]*len(years) for unique_geoid in unique_geoids]
unique_geoids = np.ndarray.flatten(np.asarray(unique_geoids))

years = [years] * len(unique_geoids)
years = np.ndarray.flatten(np.asarray(years))

tuples = list(zip(*[unique_geoids, years]))

index = pd.MultiIndex.from_tuples(tuples)

In [23]:
# Build empty dataframe with complete indexing
all_index = pd.DataFrame(index = index)
all_index.index.names = ["GEOID", "year"]

# Merge yields so that empty county/year pairings are "NaN"
yields = pd.merge(all_index, usda.reset_index(), on = ["GEOID", "year"],  how = "outer")

# Set correct indexing
yields["state_fips_code"] = yields["GEOID"].str[:2]
yields["county_code"] = yields["GEOID"].str[2:5]
yields["asd_code"] = yields["GEOID"].str[-2:]
del yields["county_name"]
yields.set_index(["GEOID", "year"], inplace = True)
yields.sort_index(inplace = True)
yields = yields.fillna(0.0)

In [24]:
# Merge with other (combined) results
yields = pd.merge(yields.reset_index(), usda_other_all, on = ["year","state_fips_code","asd_code"], how = "outer")

In [25]:
# Re-index
yields = yields.fillna(0.0)

yields["area"] = yields.apply(lambda x: x["area"] if x["area"] > 0.0 else x["OtherMean_Area"], axis = 1)

yields["Value"] = yields.apply(lambda x: x["Value"] if x["Value"] > 0.0 else x["OtherCombined_Value"], axis = 1)

del yields["OtherCombined_Value"]
del yields["OtherMean_Area"]
del yields["index"]

yields = yields.fillna(0.0)
yields.set_index(["GEOID"], inplace = True)

In [35]:
yields.head()

,year,state_fips_code,county_code,asd_code,Value,area
GEOID,,,,,,
0100140,1950,01,001,40,17.0,29600.0
0101140,1950,01,011,40,11.0,24200.0
0104740,1950,01,047,40,13.0,44200.0
0105140,1950,01,051,40,23.0,38900.0
0106340,1950,01,063,40,16.0,23900.0


In [36]:
yields.to_csv("./output/USDA_county_yields.csv")